In [1]:
import os
import numpy as np
import pandas as pd
from astropy.table import Table

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp
from lvmdrp.functions import run_quickdrp as qdrp

from lvmdrp.functions import imageMethod, rssMethod

ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")
MASTER_CON_LAMPS = {"b": "ldls", "r": "ldls", "z": "quartz"}
MASTER_ARC_LAMPS = {"b": "hgne", "r": "neon", "z": "neon"}
MASTER_ARC_LAMPS = {"b": "neon_hgne_argon_xenon", "r": "neon_hgne_argon_xenon", "z": "neon_hgne_argon_xenon"}

In [2]:
MJD = 60147
frames_table = md.get_metadata(tileid="*", mjd=MJD)
frames_table.query("imagetyp == 'bias' | imagetyp == 'dark' | imagetyp == 'flat' | imagetyp == 'arc'", inplace=True)

[INFO]: loading metadata store at /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5
[INFO]: found 543 frames in store '/home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5'
[INFO]: number of frames after filtering 543
[INFO]: total number of frames found 543


In [3]:
masters_mjd = qdrp.get_master_mjd(MJD)
masters_path = os.path.join(ORIG_MASTER_DIR, str(masters_mjd))
print(masters_path)

/home/mejia/Research/lvm/lvmdata/calib/60142


In [4]:
masters_flat = md.get_metadata(kind="master", imagetyp="flat").query("mjd == @MJD").sort_values("camera")
masters_flat

[INFO]: loading metadata store at /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/master_metadata.hdf5
[INFO]: found 120 frames in store '/home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/master_metadata.hdf5'
[INFO]: filtering by imagetyp = 'flat'
[INFO]: number of frames after filtering 18
[INFO]: total number of frames found 18


,tileid,mjd,rmjd,imagetyp,spec,camera,exptime,neon,hgne,krypton,...,argon,ldls,quartz,quality,qual,stage,status,drpqual,nframes,name
3,1111,60147,60147,flat,sp1,b1,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,4,lvm-mflat_ldls-b1
5,1111,60147,60147,flat,sp2,b2,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,4,lvm-mflat_ldls-b2
1,1111,60147,60147,flat,sp3,b3,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,4,lvm-mflat_ldls-b3
7,1111,60147,60147,flat,sp1,r1,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,4,lvm-mflat_ldls-r1
2,1111,60147,60147,flat,sp2,r2,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,4,lvm-mflat_ldls-r2
6,1111,60147,60147,flat,sp3,r3,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,4,lvm-mflat_ldls-r3
4,1111,60147,60147,flat,sp1,z1,10.0,False,False,False,...,False,False,True,excellent,0,1,0,0,4,lvm-mflat_quartz-z1
0,1111,60147,60147,flat,sp2,z2,10.0,False,False,False,...,False,False,True,excellent,0,1,0,0,4,lvm-mflat_quartz-z2
8,1111,60147,60147,flat,sp3,z3,10.0,False,False,False,...,False,False,True,excellent,0,1,0,0,4,lvm-mflat_quartz-z3


In [5]:
mtrace_paths = []
for flat in masters_flat.to_dict("records"):
    camera = flat["camera"]
    # define lamps for flats and arcs
    con_lamp = MASTER_CON_LAMPS[camera[0]]
    arc_lamp = MASTER_ARC_LAMPS[camera[0]]
    
    # define masters paths
    mflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=MJD, kind=f"mflat_{con_lamp}", camera=camera)
    mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{camera}.fits")
    mwidth_path = os.path.join(masters_path, f"lvm-mwidth-{camera}.fits")
    mwave_path = os.path.join(masters_path, f"lvm-mwave_{arc_lamp}-{camera}.fits")
    mlsf_path = os.path.join(masters_path, f"lvm-mwave_{arc_lamp}-{camera}.fits")
    # define master fiberflat path
    wflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=MJD, kind=f"wmflat_{con_lamp}", camera=camera)

    if os.path.exists(wflat_path):
        print(f"skipping {wflat_path}, file already exist")
        continue
        
    print(f"Processing {mflat_path} -> {wflat_path}")

    # TODO: run the fiber fitting extraction to remove the aperture extraction features
    # imageMethod.extract_spectra(in_image=mflat_path, out_rss=wflat_path, in_trace=mtrace_path, fwhm=mwidth_path, method="optimal", parallel=4)
    imageMethod.extract_spectra(in_image=mflat_path, out_rss=wflat_path, in_trace=mtrace_path, method="aperture", aperture=3)
    rssMethod.create_pixel_table(in_rss=wflat_path, out_rss=wflat_path, arc_wave=mwave_path, arc_fwhm=mlsf_path)

skipping /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmflat_ldls-b1.fits, file already exist
skipping /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmflat_ldls-b2.fits, file already exist
skipping /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmflat_ldls-b3.fits, file already exist
skipping /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmflat_ldls-r1.fits, file already exist
skipping /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmflat_ldls-r2.fits, file already exist
skipping /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmflat_ldls-r3.fits, file already exist
skipping /home/mejia/Research/lvm/lvmdata/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmflat_quartz-z1.fits, fi

In [9]:
wflat_paths = []
mfiberflat_paths = []
for flat in masters_flat.to_dict("records"):
    camera = flat["camera"]
    # define lamps for flats and arcs
    con_lamp = MASTER_CON_LAMPS[camera[0]]
    arc_lamp = MASTER_ARC_LAMPS[camera[0]]
    
    # define master fiberflat path
    wflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=masters_mjd, kind=f"wmflat_{con_lamp}", camera=camera)
    wflat_paths.append(wflat_path)
    
    mfiberflat_path = os.path.join(masters_path, f"lvm-mfiberflat-{camera}.fits")
    mfiberflat_paths.append(mfiberflat_path)

rssMethod.create_fiberflat(in_rsss=wflat_paths, out_rsss=mfiberflat_paths, median_box=200, gaussian_kernel=10, display_plots=True)

[INFO]: reading continuum exposure from lvm-wmflat_ldls-b1.fits
[INFO]: reading continuum exposure from lvm-wmflat_ldls-b2.fits
[INFO]: reading continuum exposure from lvm-wmflat_ldls-b3.fits
[INFO]: reading continuum exposure from lvm-wmflat_ldls-r1.fits
[INFO]: reading continuum exposure from lvm-wmflat_ldls-r2.fits
[INFO]: reading continuum exposure from lvm-wmflat_ldls-r3.fits
[INFO]: reading continuum exposure from lvm-wmflat_quartz-z1.fits
[INFO]: reading continuum exposure from lvm-wmflat_quartz-z2.fits
[INFO]: reading continuum exposure from lvm-wmflat_quartz-z3.fits
[INFO]: applying median smoothing with box size [1, 200] angstroms ([1, 348] pixels)
[INFO]: caculating normalization in full wavelength range (3504.81 - 5990.90 angstroms)
[INFO]: caculating normalization in full wavelength range (5560.01 - 7790.14 angstroms)
[INFO]: caculating normalization in full wavelength range (7343.97 - 9889.75 angstroms)
[INFO]: computing fiberflat across 1944 fibers and 4085 wavelength bi

In [7]:
# for channel in "brz":
#     wflat_paths, mfiberflat_paths = [], []
#     for flat in masters_flat.to_dict("records"):
#         camera = flat["camera"]
#         if camera[0] != channel: continue
#         con_lamp = MASTER_CON_LAMPS[camera[0]]
#         wflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=masters_mjd, kind=f"wmflat_{con_lamp}", camera=camera)
#         mfiberflat_path = os.path.join(masters_path, f"lvm-mfiberflat-{camera}.fits")
#         mfiberflat_paths.append(mfiberflat_path)
#         wflat_paths.append(wflat_path)

#     # TODO: smooth extracted spectra before computing the median spectrum
#     # rssMethod.create_fiberflat(in_rsss=wflat_paths, out_rsss=mfiberflat_paths, median_box=200, gaussian_kernel=10, display_plots=True)
#     rssMethod.create_fiberflat(in_rsss=wflat_paths, out_rsss=mfiberflat_paths, median_box=0, gaussian_kernel=10, display_plots=True)

# # cache new masters metadata into HDF5 files
# new_masters = md.extract_metadata(frames_paths=mfiberflat_paths)
# md.add_masters(new_masters)

In [8]:
# fibers = [2, 106, 37]
# select = np.isin(fibermap["fiberid"]-1, fibers)
# skw1_fibers = fibermap[~select][fibermap[~select]["ifulabel"] == "SkyW1"]["fiberid"]-1
# ske1_fibers = fibermap[~select][fibermap[~select]["ifulabel"] == "SkyE1"]["fiberid"]-1
# sci1_fibers = fibermap[~select][fibermap[~select]["ifulabel"] == "Sci1"]["fiberid"]-1

# factors = {}
# for spec in "123":
#     for cam in ["b", "r", "z"]:
#         cam = cam + spec
#         fflat = RSS()
#         fflat.loadFitsData(f"/home/mejia/Research/lvm/lvmdata/calib/60177/lvm-mfiberflat-{cam}.fits")
#         fflat._data[(fflat._mask)|(fflat._data <= 0)] = np.nan
#         sci_factor = np.nanmedian(fflat._data[sci1_fibers][:, 1000:3000])
#         print(cam, np.nanmedian(fflat._data[skw1_fibers][:, 1000:3000])/sci_factor, np.nanmedian(fflat._data[ske1_fibers][:, 1000:3000])/sci_factor)
#         factors[("SkyW", cam)] = np.nanmedian(fflat._data[skw1_fibers][:, 1000:3000])/sci_factor
#         factors[("SkyE", cam)] = np.nanmedian(fflat._data[ske1_fibers][:, 1000:3000])/sci_factor